In [1]:
import os
import time
import numpy as np
import cv2
import onnxruntime as ort
from sklearn.metrics import accuracy_score

# Directory containing the dataset
DATASET_DIR = 'datasets/American-Sign-Language-Letters-1/valid/images'
# Path to your ONNX model
MODEL_PATH = 'models/v4/best.onnx'

# Load the ONNX model
session = ort.InferenceSession(MODEL_PATH)

# Define the input name based on your model
input_name = session.get_inputs()[0].name

def preprocess_image(image_path):
    # Read and preprocess the image (resize, normalize, etc.)
    image = cv2.imread(image_path)
    image = cv2.resize(image, (640, 640))  # Adjust size according to your model
    image = image.astype(np.float32) / 255.0  # Normalize
    image = np.transpose(image, (2, 0, 1))  # Change to CHW format
    image = np.expand_dims(image, axis=0)  # Add batch dimension
    return image

def get_label_from_filename(filename):
    # Extract label from filename (e.g., "A65_xxxxx" -> "A")
    return filename.split('_')[0][0]

# Prepare lists to store true labels and predictions
true_labels = []
predictions = []
yolo_class = ["A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K", "L", "M", "N", "O", "P", "Q", "R", "S", "T", "U", "V", "W", "X", "Y", "Z"]

# Iterate over the dataset and run inference
start_time = time.time()
for filename in os.listdir(DATASET_DIR):
    if filename.endswith('.jpg') or filename.endswith('.png'):  # Adjust extensions if needed
        image_path = os.path.join(DATASET_DIR, filename)
        
        # Preprocess image
        image = preprocess_image(image_path)
        
        # Run inference
        outputs = session.run(None, {input_name: image})
        output = np.array(outputs[0])
        class_indices = np.argmax(output, axis=-1)

        predicted_label = class_indices
        # Get true label from filename
        true_label = get_label_from_filename(filename)
        
        # Append results
        true_labels.append(true_label)
        predictions.append(predicted_label)

end_time = time.time()
inference_time = end_time - start_time

# Calculate accuracy
print(true_labels)
print(predictions)

# print(f'Accuracy: {accuracy * 100:.2f}%')
print(f'Inference time: {inference_time:.2f} seconds')


['A', 'A', 'A', 'A', 'A', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'C', 'C', 'C', 'D', 'D', 'D', 'D', 'D', 'D', 'E', 'E', 'E', 'E', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'G', 'G', 'G', 'G', 'G', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'I', 'I', 'J', 'J', 'J', 'J', 'J', 'J', 'J', 'J', 'K', 'K', 'K', 'K', 'K', 'K', 'L', 'L', 'L', 'L', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'N', 'N', 'N', 'N', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'P', 'P', 'P', 'P', 'P', 'P', 'P', 'Q', 'Q', 'Q', 'Q', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'S', 'S', 'S', 'S', 'T', 'T', 'T', 'T', 'T', 'T', 'U', 'U', 'U', 'U', 'U', 'U', 'U', 'V', 'V', 'V', 'V', 'V', 'W', 'W', 'W', 'X', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Z', 'Z', 'Z', 'Z']
[array([[1199, 6327, 8129, 8149, 8328, 8168, 8190, 8170, 8271, 8189, 8242,
        8250, 8152, 8369, 8152, 8066, 8212, 8270, 8290, 8210, 8209, 8190,
        8251, 8247, 8212, 8170, 8212, 8191, 8250, 8105]], dtype=int64), array([[4159, 6377, 8027, 8148, 8269, 8228, 8270, 8190, 82